Second strategy:
    - Calculate moving average 100
    - Calculate moving average 50
    - Calculate volume mean
    - Check if the averages cross each other
    - Check if the volume at the cross are bigger than mean of volumes
    - Check the price market after 10 candles



In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import datetime as dt
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
yf.pdr_override()

start = dt.datetime(20,1,1)
end = dt.datetime.now()
ticker = 'BBAS3.SA'

df_data = pdr.get_data_yahoo(ticker, start, end, interval='1d')
df_data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,1.400000,1.426666,1.366666,1.366666,0.531419,1029600
2000-01-04,1.366666,1.370000,1.301666,1.301666,0.506145,628200
2000-01-05,1.300000,1.316666,1.253333,1.315000,0.511329,957600
2000-01-06,1.315000,1.333333,1.300000,1.333333,0.518458,410400
2000-01-07,1.333333,1.333333,1.303333,1.303333,0.506793,788400
...,...,...,...,...,...,...
2024-06-04,27.150000,27.250000,26.980000,27.230000,27.230000,18277800
2024-06-05,27.250000,27.400000,27.150000,27.290001,27.290001,14021500
2024-06-06,27.299999,27.719999,27.200001,27.650000,27.650000,13834300


In [3]:
# first you must create moving average 100
df_data['SMA100'] = df_data['Close'].rolling(window=100).mean()

# second you must create moving average 50
df_data['SMA50'] = df_data['Close'].rolling(window=50).mean()

# third you must to create volume mean
df_data['Mean Size Volume'] = df_data['Volume'].rolling(window=100).quantile(0.9)

# Fouth you must check the sign
df_data["Sign"] = np.sign(df_data["Close"] - df_data["Open"])

df_data.dropna(inplace=True)


In [4]:
# check for crossovers
cond1 = (df_data["SMA50"].shift(1) > df_data["SMA100"].shift(1)) & (df_data['SMA50'] < df_data['SMA100'])
# check if the volume are smaller than mean of volumes
cond2 = df_data["Volume"] < df_data["Mean Size Volume"]
# check if the sign are negative
cond3 = df_data["Sign"] == -1

conditions_met = df_data[cond1 & cond2 & cond3]

df_data[cond1 & cond2 & cond3]

,Open,High,Low,Close,Adj Close,Volume,SMA100,SMA50,Mean Size Volume,Sign
Date,,,,,,,,,,
2000-10-23,0.955000,0.983333,0.901666,0.916666,0.356440,792000,1.051250,1.049633,1565280.0,-1.0
2001-10-03,1.181666,1.203333,1.158333,1.166666,0.453651,1445400,1.456383,1.451966,2532060.0,-1.0
2002-07-01,1.488333,1.488333,1.418333,1.418333,0.551510,1132200,1.970033,1.967333,2001780.0,-1.0
2002-11-08,1.718333,1.748333,1.716666,1.716666,0.667515,4757400,1.572083,1.567900,7729020.0,-1.0
2005-04-05,4.883333,4.883333,4.625000,4.698333,1.826917,3065400,5.312283,5.305766,9129240.0,-1.0
2006-07-07,8.081666,8.085000,7.716666,7.833333,3.045941,141633552,9.193416,9.179033,182294058.0,-1.0
2007-10-16,14.400000,14.550000,14.300000,14.350000,5.579906,5568800,14.215617,14.205800,6875000.0,-1.0
2008-08-07,12.000000,12.085000,11.705000,11.900000,4.627240,4679400,13.267450,13.264400,13215920.0,-1.0
2010-05-18,13.750000,13.870000,13.280000,13.445000,5.527758,8537400,14.737850,14.729600,9793160.0,-1.0


In [5]:
# Passo 4: Plotar com Plotly
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, specs=[[{"secondary_y": True}], [{}]],
                    vertical_spacing=0.05, subplot_titles=('BBAS3', 'Volume'), row_width=[0.2, 0.8])

# Adicionar trace de candlestick
fig.add_trace(go.Candlestick(x=df_data.index,
                             open=df_data["Open"],
                             high=df_data["High"],
                             low=df_data["Low"],
                             close=df_data["Close"], name='Candlestick'), row=1, col=1)

# Add trace of SMA100
fig.add_trace(go.Scatter(x=df_data.index, y=df_data["SMA100"], mode='lines', name='SMA100', line=dict(color='blue')), row=1, col=1)


# Add trace of SMA50
fig.add_trace(go.Scatter(x=df_data.index, y=df_data["SMA50"], mode='lines', name='SMA50', line=dict(color='orange')), row=1, col=1)

# Add scatter trace to highlight the entry points
fig.add_trace(go.Scatter(x=conditions_met.index, y=conditions_met["Close"], 
                        mode='markers', marker=dict(size=10, color='black', symbol='triangle-up'), name='Entry Points'), row=1, col=1)

# Add trace of volume
fig.add_trace(go.Bar(x=df_data.index, y=df_data["Volume"], name='Volume'), row=2, col=1)

# Update layout
fig.update_layout(height=700, xaxis_rangeslider_visible=False, template='seaborn')
fig.update_layout(margin=dict(l=30, r=30, t=30, b=30))

fig.show()

In [6]:
df_data["Close Fut"] = df_data['Close'].shift(-6) - df_data['Close']
df_data.dropna(inplace=True)
df_data[cond1 & cond2 & cond3]['Close Fut']

/var/folders/h3/hg0brw_s1mb4rb20y5dzl_sw0000gn/T/ipykernel_14738/3650104775.py:3: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



Date
2000-10-23    0.056667
2001-10-03    0.000000
2002-07-01    0.006667
2002-11-08   -0.013333
2005-04-05   -0.031667
2006-07-07   -0.133333
2007-10-16    0.500000
2008-08-07   -0.754999
2010-05-18   -0.545000
2012-05-14    0.055000
2013-06-17   -0.865000
2014-01-15   -0.830000
2014-11-14    2.365000
2015-07-20   -0.515000
2016-07-04    0.330001
2017-12-20    0.485000
2018-05-23   -0.520000
2019-05-16    1.990000
2019-09-02    1.754999
2021-03-11    0.325001
2023-09-29    0.705000
Name: Close Fut, dtype: float64

In [7]:
df_data[cond1 & cond2 & cond3]['Close Fut'].mean()


/var/folders/h3/hg0brw_s1mb4rb20y5dzl_sw0000gn/T/ipykernel_14738/3339845820.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



0.20785721426918394

In [8]:
df_data[cond1 & cond2 & cond3]['Close Fut'].std()


/var/folders/h3/hg0brw_s1mb4rb20y5dzl_sw0000gn/T/ipykernel_14738/1294166152.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



0.8884779596221687